# Portfolio Rebalancer in Python

In [ ]:
#Steps:
#1.Define our current Portfolio (accounttype, time, ticker, shares, cost basis, price)
#2.Define our target allocation (ticker, allocation)
#3.Calculate steps needed to hit target allocation
#4.Set triggers to rebalance (time or threshold or both)
#5.Decide on optimal rebalance strategy (sell assets, invest additional assets, factor in taxable accounts vs. tax deferred)

#Phases:
#1. Rebalance an existing portfolio
#2. Rebalance new money in
#3. Factor in account types and tax consideratons
#4. Track portfolio changes using API
#5. Email alerts

#Feature Ideas:
#Calculate how to distribute new money given current allocation and target
#Calculate capital gains and losses (and their tax status LT vs. ST)
#Define allocation at asset-class / domestic-international (or other) levels
#Enable buying/selling fractional shares for asset types that allow it (ETFs, individual stocks, etc..)
#Factor in trading fees
#Convert the top-level allocation to the ticker-level allocation (see: https://docs.google.com/spreadsheets/d/1Ig0sYTdfSqcM2OHJDa7m_vRvQeAQO823s2Nw2rGmL5g/edit#gid=628577776)
#Calculate Tax Location for assets (define optimal tax location depending on the security)
#Calculate Tax Loss Harvesting (maybe a separate app)

In [ ]:
#References:
#https://nbviewer.jupyter.org/github/kdboller/pythonsp500/blob/a7066d998ff046c3cc8b26ece3b0efdf00959d57/Investment%20Portfolio%20Python%20Notebook_03_2018_blog%20example.ipynb

In [1]:
import pandas as pd
import numpy as np
import datetime
import decimal
from pandas_datareader import data as pdr
#import fix_yahoo_finance as yf
#yf.pdr_override() # <== Fix for yahoo data
from keys import tiingo_key
now = datetime.datetime.now()

In [65]:
#Settings

new_money_in = 10000
#Set our rebalance threshold
rebal_threshold = .05 #allowable allocation drift
rebal_timeframe = 180 #in days

In [2]:
#create our target allocation
columns = ['ticker','allocation_target']
positions = [['VTSAX',0.5652],
             ['VIGAX',0.0131],
             ['VSMAX',0.0066],
             ['VSEQX',0.0066],
             ['VWIGX',0.0507],
             ['VTRIX',0.0507],
             ['VTIAX',0.1521],
             ['VBTLX',0.035],
             ['VTABX',0.015],
             ['VGSLX',0.05],
             ['VNQI',0.01],
             ['VDE',0.03],
             ['GLD',0.015]]

targetalloc = pd.DataFrame(columns = columns, data = positions)
total=decimal.Decimal(targetalloc.allocation_target.sum())
if round(total,4) != 1:
    print('Target Allocation not 100% : {}'.format(int(total)))
targetalloc

,ticker,allocation_target
0,VTSAX,0.5652
1,VIGAX,0.0131
2,VSMAX,0.0066
3,VSEQX,0.0066
4,VWIGX,0.0507
5,VTRIX,0.0507
6,VTIAX,0.1521
7,VBTLX,0.0350
8,VTABX,0.0150
9,VGSLX,0.0500


In [74]:
#lookup table for account type abbreviations
accounttypes = {'TAXB':'Taxable Brokerage', '401K':'401k', 'RIRA':'Roth-IRA', 'TIRA':'Traditional-IRA'}
#upload our current portfolio
columns = ['accounttype','lastrebaldate','ticker','basisdate','costbasis','shares']
positions = [['TAXB','2018-11-16','VTSAX','2012-10-27',120.20,2],
             ['401K','2018-11-16','VIGAX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VSMAX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VSEQX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VWIGX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VTRIX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VTIAX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VBTLX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VTABX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VGSLX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VNQI','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VDE','2017-1-02',56.70,14.5],
            ['401K','2018-11-16','AAPL','2017-1-02',56.70,14.5]]

start_port = pd.DataFrame(columns = columns, data = positions)
start_port.lastrebaldate = pd.to_datetime(start_port.lastrebaldate)
start_port.basisdate = pd.to_datetime(start_port.basisdate)

In [4]:
#Pull list of distinct tickers we care about
tickers = set(targetalloc.ticker.unique().tolist()+start_port.ticker.unique().tolist())

#Next we pull the latest prices from Tiingo (YahooFinance is buggy, and IEX does not contain mutual fund data)
#Tiingo limits for free API: 500 unique tickers ever, 500 requests/hr, 20,000 requests/day
#https://pandas-datareader.readthedocs.io/en/latest/remote_data.html#tiingo
#Tiingo API key required: set 'tiingo_key' value in python file called 'keys.py' in same directory as this script
now = datetime.datetime.now()
yesterday = now - datetime.timedelta(3) #avoids weekends with no data - need better weekend detection
start = datetime.datetime(yesterday.year, yesterday.month, yesterday.day)
end = datetime.datetime(now.year, now.month, now.day)

bad_tickers = []
for i, t in enumerate(tickers):
    try:
        if i==0:
            ohlc = pdr.get_data_tiingo(t, api_key=tiingo_key).tail(1).close
        else:
            ohlc = ohlc.append(pdr.get_data_tiingo(t, api_key=tiingo_key).tail(1).close)
    except:
        bad_tickers.append(t)
        
#print(bad_tickers)
ohlc = ohlc.to_frame(name='close')

#drop our date index since its only the latest data
ohlc2=ohlc.reset_index(level=1, drop=True)

In [5]:
ohlc2

,close
symbol,
VGSLX,116.36
VSMAX,68.00
VDE,86.90
VTRIX,34.87
VTIAX,26.13
GLD,118.09
AAPL,168.49
VNQI,54.59
VTABX,21.97


In [82]:
#Concatenate target allocation and latest prices with our portfolio
start_port_c = pd.merge(start_port, targetalloc, on ='ticker', how ='outer')
final_port = pd.merge(start_port_c, ohlc2, left_on ='ticker', right_index = True, how = 'left')
#Set target to zero for tickers held but not present in our target allocation, set initial basisdate and costbasis for new securities entering the portfolio
final_port.fillna(value = {'allocation_target':0.0,'shares':0.0,'basisdate':pd.to_datetime(now.strftime("%Y-%m-%d")),'costbasis':final_port.close},inplace = True)

In [83]:
#Calc holding values and current allocation
final_port['value'] = final_port.close * final_port.shares #calculate value as price x shares
final_port.loc[final_port.value.isna() & final_port.shares.isna(),['value']]=0.0 #for securities not currently held but in our target (and close price failed to return), establish zero value
final_port['allocation'] = final_port.value / final_port.value.sum()
final_port['correction'] = final_port.allocation_target - final_port.allocation

In [84]:
total_val = final_port.value.sum()
#Account for new money in
final_port['new_money_in'] = new_money_in * final_port.allocation_target

final_port['value_chg'] = (total_val * final_port.correction) + final_port.new_money_in
final_port['shares_chg'] = final_port.value_chg / final_port.close
final_port.loc[final_port.value_chg.isna() & final_port.shares > 0,['shares_chg']]=-final_port.shares #sell all shares of securities not in our target portfolio

#round off shares to whole numbers, except when we are fully exiting a position
final_port['shares_chg_round'] = final_port.shares_chg
final_port = final_port.astype({'shares_chg_round': int})
final_port['final_shares_chg'] = final_port.shares_chg
final_port.loc[final_port.shares_chg+final_port.shares!=0,['final_shares_chg']]=final_port.shares_chg_round*1.0
final_port.drop(['shares_chg_round'],axis=1,inplace=True)

#create timedelta int column
final_port['timedelta'] = (final_port.lastrebaldate - pd.to_datetime(now.strftime("%Y-%m-%d"))).dt.days
final_port.timedelta.fillna(0,inplace=True)

final_port['rebal_flag_thresh'] = np.where((abs(final_port.correction)<=rebal_threshold) & (final_port.allocation > 0),0,1)
final_port['rebal_flag_time'] = np.where(final_port.timedelta >= rebal_timeframe,1,0)
final_port['rebal_flag'] = np.where(final_port.rebal_flag_thresh + final_port.rebal_flag_time >= 1,1,0)

#cleanup
#final_port.drop(columns=['timedelta'],inplace=True)

#These rows do not meet our rebalance threshold or timeframe cutoff - so we should set their change values to 0
final_port.loc[final_port.rebal_flag==0, ['value_chg','shares_chg','final_shares_chg']] = 0

final_port['new_shares'] = (final_port.shares + final_port.final_shares_chg)
final_port['new_value'] = final_port.new_shares * final_port.close #due to share rounding, there will be slight variance vs. portfolio starting value
final_port['new_value_chg'] = final_port.final_shares_chg * final_port.close

In [85]:
final_port

,accounttype,lastrebaldate,ticker,basisdate,costbasis,shares,allocation_target,close,value,allocation,...,value_chg,shares_chg,final_shares_chg,timedelta,rebal_flag_thresh,rebal_flag_time,rebal_flag,new_shares,new_value,new_value_chg
0,TAXB,2018-11-16,VTSAX,2012-10-27,120.20,2.0,0.5652,65.69,131.380,0.012442,...,11488.597886,174.891123,174.0,-23.0,1,0,1,176.0,11561.440,11430.060
1,401K,2018-11-16,VIGAX,2017-01-02,56.70,14.5,0.0131,72.55,1051.975,0.099628,...,-782.651379,-10.787752,-10.0,-23.0,1,0,1,4.5,326.475,-725.500
2,401K,2018-11-16,VSMAX,2017-01-02,56.70,14.5,0.0066,68.00,986.000,0.093380,...,-850.310237,-12.504562,-12.0,-23.0,1,0,1,2.5,170.000,-816.000
3,401K,2018-11-16,VSEQX,2017-01-02,56.70,14.5,0.0066,31.71,459.795,0.043545,...,0.000000,0.000000,0.0,-23.0,0,0,0,14.5,459.795,0.000
4,401K,2018-11-16,VWIGX,2017-01-02,56.70,14.5,0.0507,27.22,394.690,0.037379,...,0.000000,0.000000,0.0,-23.0,0,0,0,14.5,394.690,0.000
5,401K,2018-11-16,VTRIX,2017-01-02,56.70,14.5,0.0507,34.87,505.615,0.047884,...,0.000000,0.000000,0.0,-23.0,0,0,0,14.5,505.615,0.000
6,401K,2018-11-16,VTIAX,2017-01-02,56.70,14.5,0.1521,26.13,378.885,0.035882,...,2748.147266,105.172111,105.0,-23.0,1,0,1,119.5,3122.535,2743.650
7,401K,2018-11-16,VBTLX,2017-01-02,56.70,14.5,0.0350,10.36,150.220,0.014227,...,0.000000,0.000000,0.0,-23.0,0,0,0,14.5,150.220,0.000
8,401K,2018-11-16,VTABX,2017-01-02,56.70,14.5,0.0150,21.97,318.565,0.030170,...,0.000000,0.000000,0.0,-23.0,0,0,0,14.5,318.565,0.000
9,401K,2018-11-16,VGSLX,2017-01-02,56.70,14.5,0.0500,116.36,1687.220,0.159789,...,-659.267250,-5.665755,-5.0,-23.0,1,0,1,9.5,1105.420,-581.800


In [58]:
#When rebal_flag = 1, We need to loop through the dframe iteratively and make our largest sales then our biggest buys. 

#First we sort the df by rebal value
final_port.sort_values(by='new_value_chg',ascending=True,inplace=True)

bal = 0
#Next we iteratively sell/buy to get our final execution strategy
for index, row in final_port.iterrows():
    bal-=row.new_value_chg
    if bal > 0:
        final_port.loc[index,'execution_value'] = final_port[index].new_value
        final_port.loc[index,'execution_shares_chg'] = final_port[index].final_shares_chg
    else:
        final_port.loc[index,'execution_value'] = final_port[index].new_value + bal
        final_port.loc[index,'execution_shares_chg'] = int((final_port[index].new_value + bal)/final_port[index].close)
    print('Evaluating {} {}, new bal is {}'.format(row.ticker,row.new_value_chg,bal))
    

final_port['execution_value_chg'] = final_port.execution_shares_chg * final_port.close
final_port['execution_alloation'] = final_port.execution_value / final_port.execution_value.sum()
    
    #WE HAVE AN ISSUE WITH MONEY-IN POTENTIALLY GIVING INCORRECT SALES NUMBERS - CHECK THIS.

Evaluating AAPL -4717.72, new bal is 4717.72
Evaluating VGSLX -2210.84, new bal is 6928.56
Evaluating VDE -1824.9, new bal is 8753.460000000001
Evaluating VSMAX -1768.0, new bal is 10521.460000000001
Evaluating VIGAX -1741.1999999999998, new bal is 12262.66
Evaluating VNQI -1310.16, new bal is 13572.82
Evaluating VSEQX 0.0, new bal is 13572.82
Evaluating VWIGX 0.0, new bal is 13572.82
Evaluating VTRIX 0.0, new bal is 13572.82
Evaluating VBTLX 0.0, new bal is 13572.82
Evaluating VTABX 0.0, new bal is 13572.82
Evaluating GLD 236.18, new bal is 13336.64
Evaluating VTIAX 2377.83, new bal is 10958.81
Evaluating VTSAX 11298.68, new bal is -339.8700000000008


,accounttype,lastrebaldate,ticker,basisdate,costbasis,shares,allocation_target,close,value,allocation,...,timedelta,rebal_flag_thresh,rebal_flag_time,rebal_flag,new_shares,new_value,execution_value,execution_shares_chg,execution_value_chg,execution_allocation
0,TAXB,2018-11-16,VTSAX,2012-10-27,120.20,2.0,0.5652,65.69,131.380,0.012442,...,-23.0,1,0,1,90.0,5912.100,5386.580,82.0,5386.580,0.519674
1,401K,2018-11-16,VIGAX,2017-01-02,56.70,14.5,0.0131,72.55,1051.975,0.099628,...,-23.0,1,0,1,2.5,181.375,181.375,-12.0,-870.600,0.017498
2,401K,2018-11-16,VSMAX,2017-01-02,56.70,14.5,0.0066,68.00,986.000,0.093380,...,-23.0,1,0,1,1.5,102.000,102.000,-13.0,-884.000,0.009841
3,401K,2018-11-16,VSEQX,2017-01-02,56.70,14.5,0.0066,31.71,459.795,0.043545,...,-23.0,0,0,0,14.5,459.795,459.795,0.0,0.000,0.044359
4,401K,2018-11-16,VWIGX,2017-01-02,56.70,14.5,0.0507,27.22,394.690,0.037379,...,-23.0,0,0,0,14.5,394.690,394.690,0.0,0.000,0.038078
5,401K,2018-11-16,VTRIX,2017-01-02,56.70,14.5,0.0507,34.87,505.615,0.047884,...,-23.0,0,0,0,14.5,505.615,505.615,0.0,0.000,0.048780
6,401K,2018-11-16,VTIAX,2017-01-02,56.70,14.5,0.1521,26.13,378.885,0.035882,...,-23.0,1,0,1,60.5,1580.865,1580.865,46.0,1201.980,0.152515
7,401K,2018-11-16,VBTLX,2017-01-02,56.70,14.5,0.0350,10.36,150.220,0.014227,...,-23.0,0,0,0,14.5,150.220,150.220,0.0,0.000,0.014493
8,401K,2018-11-16,VTABX,2017-01-02,56.70,14.5,0.0150,21.97,318.565,0.030170,...,-23.0,0,0,0,14.5,318.565,318.565,0.0,0.000,0.030734
9,401K,2018-11-16,VGSLX,2017-01-02,56.70,14.5,0.0500,116.36,1687.220,0.159789,...,-23.0,1,0,1,5.5,639.980,639.980,-9.0,-1047.240,0.061743


In [35]:
display_port = final_port[['accounttype','lastrebaldate','ticker','costbasis','shares','close','allocation','allocation_target','execution_value','execution_shares_chg','execution_value_chg','execution_allocation']]
display_port

,accounttype,lastrebaldate,ticker,costbasis,shares,close,allocation,allocation_target,execution_value,execution_shares_chg,execution_value_chg,execution_allocation
0,TAXB,2018-11-16,VTSAX,120.20,2.0,65.69,0.012442,0.5652,5386.580,82.0,5386.580,0.519674
1,401K,2018-11-16,VIGAX,56.70,14.5,72.55,0.099628,0.0131,181.375,-12.0,-870.600,0.017498
2,401K,2018-11-16,VSMAX,56.70,14.5,68.00,0.093380,0.0066,102.000,-13.0,-884.000,0.009841
3,401K,2018-11-16,VSEQX,56.70,14.5,31.71,0.043545,0.0066,459.795,0.0,0.000,0.044359
4,401K,2018-11-16,VWIGX,56.70,14.5,27.22,0.037379,0.0507,394.690,0.0,0.000,0.038078
5,401K,2018-11-16,VTRIX,56.70,14.5,34.87,0.047884,0.0507,505.615,0.0,0.000,0.048780
6,401K,2018-11-16,VTIAX,56.70,14.5,26.13,0.035882,0.1521,1580.865,46.0,1201.980,0.152515
7,401K,2018-11-16,VBTLX,56.70,14.5,10.36,0.014227,0.0350,150.220,0.0,0.000,0.014493
8,401K,2018-11-16,VTABX,56.70,14.5,21.97,0.030170,0.0150,318.565,0.0,0.000,0.030734
9,401K,2018-11-16,VGSLX,56.70,14.5,116.36,0.159789,0.0500,639.980,-9.0,-1047.240,0.061743
